# Parte 1

Esta primeira parte do trabalho tem dois objetivos principais. O primeiro é verificar alguns parâmetros do hardware e do software utilizado e o segundo é que os estudantes preparem e se familiarizar com o ambiente do PostgreSQL.

## Tarefa 1 – Identificação do Sistema

Identifique o sistema que será usado para os experimentos, incluindo informações sobre o hardware e o Sistema Operacional utilizado. Sobre o tipo de processador, quantidade de memória RAM, tamanho do disco. Devem ser também apresentadas informações sobre as caches existentes. Sobre o Sistema Operacional, que devem ser Linux, incluir informações sobre qual a distribuição usada, versão do sistema, versão do Kernel, etc.

In [6]:
# Source - https://stackoverflow.com/a
# Posted by Yogesh Jog, modified by community. See post 'Timeline' for change history
# Retrieved 2025-11-22, License - CC BY-SA 4.0

#Shamelessly combined from google and other stackoverflow like sites to form a single function

import platform,socket,re,uuid,json,psutil,logging

def getSystemInfo():
    try:
        info={}
        info['platform']=platform.system()
        info['platform-release']=platform.release()
        info['platform-version']=platform.version()
        info['architecture']=platform.machine()
        info['hostname']=socket.gethostname()
        info['ip-address']=socket.gethostbyname(socket.gethostname())
        info['mac-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['processor']=platform.processor()
        info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

json.loads(getSystemInfo())


{'platform': 'Linux',
 'platform-release': '6.6.87.2-microsoft-standard-WSL2',
 'platform-version': '#1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025',
 'architecture': 'x86_64',
 'hostname': 'DESKTOP-7HG5CHA',
 'ip-address': '127.0.1.1',
 'mac-address': '61:e5:17:cd:ec:6a',
 'processor': 'x86_64',
 'ram': '8 GB'}

In [7]:
print("--- Informações da CPU e Caches ---")
!lscpu | grep -E 'Model name|Socket|Thread|Core|Cache'

print("\n--- Informações de Memória RAM ---")
!free -h

print("\n--- Informações de Disco ---")
!lsblk -o NAME,SIZE,TYPE,MOUNTPOINT
!df -h /

print("\n--- Informações do Sistema Operacional ---")
!cat /etc/*release
!uname -r

--- Informações da CPU e Caches ---
Model name:                           Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Thread(s) per core:                   2
Core(s) per socket:                   4
Socket(s):                            1

--- Informações de Memória RAM ---
               total        used        free      shared  buff/cache   available
Mem:           7.7Gi       2.2Gi       1.0Gi       142Mi       4.4Gi       5.2Gi
Swap:          2.0Gi          0B       2.0Gi

--- Informações de Disco ---
NAME   SIZE TYPE MOUNTPOINT
sda  388.4M disk 
sdb    186M disk 
sdc      2G disk [SWAP]
sdd      1T disk /mnt/wslg/distro
Filesystem      Size  Used Avail Use% Mounted on
/dev/sdd       1007G  6.3G  950G   1% /

--- Informações do Sistema Operacional ---
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.2 LTS"
PRETTY_NAME="Ubuntu 22.04.2 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.2 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=

In [ ]:
!pip install py-cpuinfo

In [8]:
# Source - https://stackoverflow.com/a/70810265
# Posted by 24_saurabh sharma
# Retrieved 2025-11-22, License - CC BY-SA 4.0

import psutil
import platform
from datetime import datetime
import cpuinfo
import socket
import uuid
import re


def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

def System_information():
    print("="*40, "System Information", "="*40)
    uname = platform.uname()
    print(f"System: {uname.system}")
    print(f"Node Name: {uname.node}")
    print(f"Release: {uname.release}")
    print(f"Version: {uname.version}")
    print(f"Machine: {uname.machine}")
    print(f"Processor: {uname.processor}")
    print(f"Processor: {cpuinfo.get_cpu_info()['brand_raw']}")
    print(f"Ip-Address: {socket.gethostbyname(socket.gethostname())}")
    print(f"Mac-Address: {':'.join(re.findall('..', '%012x' % uuid.getnode()))}")


    # Boot Time
    print("="*40, "Boot Time", "="*40)
    boot_time_timestamp = psutil.boot_time()
    bt = datetime.fromtimestamp(boot_time_timestamp)
    print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")


    # print CPU information
    print("="*40, "CPU Info", "="*40)
    # number of cores
    print("Physical cores:", psutil.cpu_count(logical=False))
    print("Total cores:", psutil.cpu_count(logical=True))
    # CPU frequencies
    cpufreq = psutil.cpu_freq()
    print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
    print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
    print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
    # CPU usage
    print("CPU Usage Per Core:")
    for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
        print(f"Core {i}: {percentage}%")
    print(f"Total CPU Usage: {psutil.cpu_percent()}%")


    # Memory Information
    print("="*40, "Memory Information", "="*40)
    # get the memory details
    svmem = psutil.virtual_memory()
    print(f"Total: {get_size(svmem.total)}")
    print(f"Available: {get_size(svmem.available)}")
    print(f"Used: {get_size(svmem.used)}")
    print(f"Percentage: {svmem.percent}%")



    print("="*20, "SWAP", "="*20)
    # get the swap memory details (if exists)
    swap = psutil.swap_memory()
    print(f"Total: {get_size(swap.total)}")
    print(f"Free: {get_size(swap.free)}")
    print(f"Used: {get_size(swap.used)}")
    print(f"Percentage: {swap.percent}%")



    # Disk Information
    print("="*40, "Disk Information", "="*40)
    print("Partitions and Usage:")
    # get all disk partitions
    partitions = psutil.disk_partitions()
    for partition in partitions:
        print(f"=== Device: {partition.device} ===")
        print(f"  Mountpoint: {partition.mountpoint}")
        print(f"  File system type: {partition.fstype}")
        try:
            partition_usage = psutil.disk_usage(partition.mountpoint)
        except PermissionError:
            # this can be catched due to the disk that
            # isn't ready
            continue
        print(f"  Total Size: {get_size(partition_usage.total)}")
        print(f"  Used: {get_size(partition_usage.used)}")
        print(f"  Free: {get_size(partition_usage.free)}")
        print(f"  Percentage: {partition_usage.percent}%")
    # get IO statistics since boot
    disk_io = psutil.disk_io_counters()
    print(f"Total read: {get_size(disk_io.read_bytes)}")
    print(f"Total write: {get_size(disk_io.write_bytes)}")

    ## Network information
    print("="*40, "Network Information", "="*40)
    ## get all network interfaces (virtual and physical)
    if_addrs = psutil.net_if_addrs()
    for interface_name, interface_addresses in if_addrs.items():
        for address in interface_addresses:
            print(f"=== Interface: {interface_name} ===")
            if str(address.family) == 'AddressFamily.AF_INET':
                print(f"  IP Address: {address.address}")
                print(f"  Netmask: {address.netmask}")
                print(f"  Broadcast IP: {address.broadcast}")
            elif str(address.family) == 'AddressFamily.AF_PACKET':
                print(f"  MAC Address: {address.address}")
                print(f"  Netmask: {address.netmask}")
                print(f"  Broadcast MAC: {address.broadcast}")
    ##get IO statistics since boot
    net_io = psutil.net_io_counters()
    print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
    print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")


if __name__ == "__main__":

    System_information()


======================================== System Information ========================================
System: Linux
Node Name: DESKTOP-7HG5CHA
Release: 6.6.87.2-microsoft-standard-WSL2
Version: #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
Machine: x86_64
Processor: x86_64
Processor: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
Ip-Address: 127.0.1.1
Mac-Address: 61:e5:17:cd:ec:6a
======================================== Boot Time ========================================
Boot Time: 2025/11/24 14:53:38
======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 0.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 2808.00Mhz
CPU Usage Per Core:
Core 0: 3.0%
Core 1: 6.0%
Core 2: 3.0%
Core 3: 0.0%
Core 4: 1.0%
Core 5: 0.0%
Core 6: 1.0%
Core 7: 0.0%
Total CPU Usage: 8.9%
======================================== Memory Information ========================================
Total: 7.69GB
Available: 5.15GB
Used: 2.54GB


## Tarefa 2 - Verificação de parâmetros de armazenamento

a) Verifique no disco que será usado para os experimentos no laboratório os seguintes parâmetros: Nr. de superfícies, cilindros, setores por trilha, velocidade de rotação, latência rotacional; tempos de seek médio, máximo e mínimo; tempo para a próxima trilha; e taxa de transferência.

Questão a revisar:

O colab não é compatível com a maior parte dos comandos de pacotes do linux para verificação dessas informações. O disco existe, mas ele fica escondido do usuário, não permitindo manipulação direta pelo usuário.

Vai ser preciso testar em uma máquina local.

- https://www.cyberciti.biz/faq/find-hard-disk-hardware-specs-on-linux/
- https://askubuntu.com/questions/332949/what-command-do-i-use-to-find-physical-disk-size
- https://askubuntu.com/questions/182446/how-do-i-view-all-available-hdds-partitions

In [ ]:
!sudo smartctl -i /dev/sda

In [ ]:
!sudo fdisk -l /dev/sda

In [ ]:
!lsblk
!lsblk /dev/DISK
!lsblk /dev/sda

In [ ]:
!sudo lsblk -o NAME,FSTYPE,SIZE,MOUNTPOINT,LABEL

In [ ]:
# Aparentemente isso funciona localmente mas não no colaboratory
!sudo lshw -class disk -class storage #| grep -E -i "product|vendor|size|bus info"

In [ ]:
# Necessário para assegurar que as bibliotecas necessárias estejam instaladas no SO.
!sudo apt-get install -y util-linux hdparm smartmontools

In [ ]:
!sudo hdparm -I /dev/sda

In [ ]:
!lshw -short -C disk

In [ ]:
!fdisk -l | grep '^Disk /dev/'
!fdisk -l | grep '^Disk /dev/' | egrep -v '/dev/(loop|mapper|md)'

In [ ]:
!smartctl -d ata -a -i /dev/sda

In [ ]:
!sudo /sbin/fdisk -l

In [ ]:
!sudo /usr/sbin/fdisk -l

In [ ]:
!sudo apt-get install -y util-linux parted

In [ ]:
!sudo parted -l

In [ ]:
!sudo parted /dev/sda unit s print

In [ ]:
!sudo lsblk -o NAME,FSTYPE,SIZE,MOUNTPOINT,LABEL

In [ ]:
!lsblk -o NAME,SIZE,PHY-SEC,LOG-SEC,ROTA,SCHED,TYPE /dev/sda

b) Utilizando o comando “stat” do Linux, verifique os parâmetros dos parâmetros de S.O. que serão utilizados para o disco.

In [ ]:
!stat -f .

c) Verifique o tamanho de bloco utilizado e mostre como alterar o tamanho dos blocos

In [ ]:
!sudo blockdev --getbsz /dev/sda

Referências para fazer essa questão em um computador localmente:

- https://unix.stackexchange.com/questions/145241/how-to-set-block-size-using-blockdev-command
- https://askubuntu.com/questions/1372387/resize-blocks-of-linux-filesystem
- https://stackoverflow.com/questions/67088392/how-to-change-block-size-on-xfs

## Tarefa 3 – Geração de um BD para testes

  Essa tarefa consiste na criação de banco de dados e povoamento destas tabelas com dados sintéticos. Para a definição do esquema das tabelas e os dados a serem carregados usaremos a especificação e os utilitários fornecidos pelos [Benchmark TPC-H](http://www.tpc.org/tpch/).
  
  Para geração do BD no PostgreSQL, siga as instruções disponíveis em https://github.com/foliveirafilho/tpch-pgsql.

### Comandos de setup de ambiente

In [ ]:
!sudo apt update

In [ ]:
!sudo apt install -y python3 python3-pip build-essential unzip postgresql postgresql-contrib libpq-dev

In [ ]:
!pip install jupyter ipython pandas psycopg2 sqlalchemy matplotlib

In [ ]:
!python3-venv
!source venv/bin/activate

In [ ]:
!service postgresql start

In [ ]:
# Comando pra configurar usuário do postgree
!sudo -u postgres psql

In [ ]:
CREATE USER icomp WITH ENCRYPTED PASSWORD 'icomp123';
CREATE DATABASE icomp;
GRANT ALL PRIVILEGES ON DATABASE icomp TO icomp;
ALTER DATABASE icomp OWNER TO icomp;
\q

### Construindo o tpch-dbgen


In [1]:
!git clone https://github.com/foliveirafilho/tpch-pgsql.git
%cd tpch-pgsql

!wget -q https://github.com/electrum/tpch-dbgen/archive/32f1c1b92d1664dba542e927d23d86ffa57aa253.zip -O tpch-dbgen.zip
!unzip -q tpch-dbgen.zip && mv tpch-dbgen-32f1c1b92d1664dba542e927d23d86ffa57aa253 tpch-dbgen && rm tpch-dbgen.zip

!pip3 install -r requirements.txt

Cloning into 'tpch-pgsql'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 160 (from 1)
Receiving objects: 100% (164/164), 774.38 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/mnt/c/Users/capuc/Desktop/BD/Trabalho 3/BD_-_Trabalho_3_-_Tarefas_diversas/tpch-pgsql


/mnt/c/Users/capuc/Desktop/BD/Trabalho 3/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Prepara os dados
!python3 tpch_pgsql.py --host localhost --username icomp --password icomp123 --dbname icomp --dbgen-dir ./tpch-dbgen --scale 1 prepare

gcc -g -DDBNAME=\"dss\" -DLINUX -DORACLE -DTPCH -DRNG_TEST -D_FILE_OFFSET_BITS=64   -c -o build.o build.c
gcc -g -DDBNAME=\"dss\" -DLINUX -DORACLE -DTPCH -DRNG_TEST -D_FILE_OFFSET_BITS=64   -c -o driver.o driver.c
driver.c: In function ‘gen_tbl’:
driver.c:368:62: warning: format ‘%ld’ expects argument of type ‘long int’, but argument 3 has type ‘long long int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
  368 |                        printf("\nSeeds for %s at rowcount %ld\n", tdefs[tnum].comment, i);
      |                                                           ~~^                          ~
      |                                                             |                          |
      |                                                             long int                   long long int
      |                                                           %lld
gcc -g -DDBNAME=\"dss\" -DLINUX -DORACLE -DTPCH -DRNG_TEST -D_FILE_OFF

In [3]:
# Carrega os dados
!python3 tpch_pgsql.py --host localhost --username icomp --password icomp123 --dbname icomp --dbgen-dir ./tpch-dbgen --scale 1 load

dropped existing tables
cleaned database icomp
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.162002
load_data: 0:00:38.666177
index_tables: 0:00:22.610098
======================= End Results ========================


In [5]:
# Comando de teste
!PGPASSWORD=icomp123 psql -h localhost -U icomp -d icomp -c "SELECT count(*) FROM lineitem;"

  count  
---------
 6001215
(1 row)



## Tarefa 4 – Execução de consultas

A segunda tarefa deste trabalho consiste em executar e analisar um conjunto de consultas analíticas do benchmark TPC-H sobre o banco de dados gerado na Tarefa 1, observando o comportamento do PostgreSQL em diferentes tipos de operações (filtros, junções, agregações e ordenações). O objetivo é compreender como o otimizador escolhe planos de execução e medir o tempo de resposta de consultas típicas de um sistema de apoio à decisão.

O TPC-H define 22 consultas padronizadas (Q1 a Q22), mas para este trabalho será utilizado um subconjunto representativo composto por 8 consultas que abrangem diferentes padrões de acesso.

Consultas a executar: Q1, Q3, Q5, Q6, Q7, Q9, Q10 e Q12. As consultas originais podem ser obtidos no repositório oficial do benchmark. Essas consultas devem ser executadas no mesmo banco TPC-H criado na Tarefa 1, utilizando o PostgreSQL.

Para cada consulta:

- Execute o comando SQL completo no PostgreSQL.
- Mostre as 10 primeiras linhas do resultado (ou um trecho representativo).
- Execute o comando EXPLAIN ANALYZE antes da consulta para exibir o plano de execução e o tempo real medido O comando EXPLAIN ANALYZE mostra as etapas internas do plano escolhido pelo otimizador, o número de linhas processadas e o tempo de cada operação.
- Registre o tempo total de execução e comente brevemente o tipo de acesso observado (por exemplo, varredura sequencial, uso de índice, junções em hash, etc.).

In [17]:
sql_q1 = """
select
    l_returnflag,
    l_linestatus,
    sum(l_quantity) as sum_qty,
    sum(l_extendedprice) as sum_base_price,
    sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
    sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
    avg(l_quantity) as avg_qty,
    avg(l_extendedprice) as avg_price,
    avg(l_discount) as avg_disc,
    count(*) as count_order
from
    lineitem
where
    l_shipdate <= date('1998-12-01') - interval '90' day
group by
    l_returnflag,
    l_linestatus
order by
    l_returnflag,
    l_linestatus;
"""

In [ ]:
sql_q3 = """
select
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    o_orderdate,
    o_shippriority
from
    customer,
    orders,
    lineitem
where
    c_mktsegment = 'BUILDING'
    and c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and o_orderdate < date('1995-03-15')
    and l_shipdate > date('1995-03-15')
group by
    l_orderkey,
    o_orderdate,
    o_shippriority
order by
    revenue desc,
    o_orderdate
limit 10;
"""

In [ ]:
sql_q5 = """
select
    n_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue
from
    customer,
    orders,
    lineitem,
    supplier,
    nation,
    region
where
    c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and l_suppkey = s_suppkey
    and c_nationkey = s_nationkey
    and s_nationkey = n_nationkey
    and n_regionkey = r_regionkey
    and r_name = 'ASIA'
    and o_orderdate >= date('1994-01-01')
    and o_orderdate < date('1994-01-01') + interval '1' year
group by
    n_name
order by
    revenue desc;
"""

In [ ]:
sql_q6 = """
select
    sum(l_extendedprice * l_discount) as revenue
from
    lineitem
where
    l_shipdate >= date('1994-01-01')
    and l_shipdate < date('1994-01-01' + interval '1' year)
    and l_discount between 0.06 - 0.01 and 0.06 + 0.01
    and l_quantity < 24;
"""

In [ ]:
sql_q7 = """
select
    supp_nation,
    cust_nation,
    l_year,
    sum(volume) as revenue
from
    (
        select
            n1.n_name as supp_nation,
            n2.n_name as cust_nation,
            extract(year from l_shipdate) as l_year,
            l_extendedprice * (1 - l_discount) as volume
        from
            supplier,
            lineitem,
            orders,
            customer,
            nation n1,
            nation n2
        where
            s_suppkey = l_suppkey
            and o_orderkey = l_orderkey
            and c_custkey = o_custkey
            and s_nationkey = n1.n_nationkey
            and c_nationkey = n2.n_nationkey
            and (
                (n1.n_name = 'FRANCE' and n2.n_name = 'GERMANY')
                or (n1.n_name = 'GERMANY' and n2.n_name = 'FRANCE')
            )
            and l_shipdate between date('1995-01-01') and date('1996-12-31')
    ) as shipping
group by
    supp_nation,
    cust_nation,
    l_year
order by
    supp_nation,
    cust_nation,
    l_year;
"""

In [ ]:
sql_q9 = """
select
    nation,
    o_year,
    sum(amount) as sum_profit
from
    (
        select
            n_name as nation,
            extract(year from o_orderdate) as o_year,
            l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity as amount
        from
            part,
            supplier,
            lineitem,
            partsupp,
            orders,
            nation
        where
            s_suppkey = l_suppkey
            and ps_suppkey = l_suppkey
            and ps_partkey = l_partkey
            and p_partkey = l_partkey
            and o_orderkey = l_orderkey
            and s_nationkey = n_nationkey
            and p_name like '%green%'
    ) as profit
group by
    nation,
    o_year
order by
    nation,
    o_year desc;
"""

In [ ]:
sql_q10 = """
select
    c_custkey,
    c_name,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    c_acctbal,
    n_name,
    c_address,
    c_phone,
    c_comment
from
    customer,
    orders,
    lineitem,
    nation
where
    c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and o_orderdate >= date('1993-10-01')
    and o_orderdate < date('1993-10-01') + interval '3' month
    and l_returnflag = 'R'
    and c_nationkey = n_nationkey
group by
    c_custkey,
    c_name,
    c_acctbal,
    c_phone,
    n_name,
    c_address,
    c_comment
order by
    revenue desc
limit 20;
"""

In [ ]:
sql_q12 = """
select
    l_shipmode,
    sum(case
        when o_orderpriority = '1-URGENT'
            or o_orderpriority = '2-HIGH'
            then 1
        else 0
    end) as high_line_count,
    sum(case
        when o_orderpriority <> '1-URGENT'
            and o_orderpriority <> '2-HIGH'
            then 1
        else 0
    end) as low_line_count
from
    orders,
    lineitem
where
    o_orderkey = l_orderkey
    and l_shipmode in ('MAIL', 'SHIP')
    and l_commitdate < l_receiptdate
    and l_shipdate < l_commitdate
    and l_receiptdate >= date('1994-01-01')
    and l_receiptdate < date('1994-01-01') + interval '1' year
group by
    l_shipmode
order by
    l_shipmode;
"""

In [13]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 9.7 MB/s eta 0:00:00a 0:00:01


In [14]:
import psycopg2
import pandas as pd

In [10]:
conn_params = {
    "host": "localhost",
    "database": "icomp",
    "user": "icomp",
    "password": "icomp123"
}

In [ ]:
conn = psycopg2.connect(**conn_params)
print("Conectado com sucesso ao postgree!")

Conectado com sucesso ao postgree!


In [15]:
cursor = conn.cursor()

In [18]:
df = pd.read_sql(sql_q1, conn)
display(df.head(10))

/tmp/ipykernel_22579/153316431.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_q1, conn)


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
1,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
2,N,O,74476040.0,1.117017e+11,1.061182e+11,1.103670e+11,25.502227,38249.117989,0.049997,2920374
3,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870


In [19]:
conn.close()
print("Conexão ao postgree fechada com sucesso!")

Conexão ao postgree fechada com sucesso!


# Parte 2

O objetivo desta parte do trabalho é analisar o comportamento dos índices das tabelas do SGBD através do exame e análise das tabelas de estatísticas para consultas SQL sobre uma tabela criada com dados aleatórios.

# Parte 3

O objetivo desta parte do trabalho é estudar o comportamento dos otimizadores de consulta dos SGBDs através do exame e análise dos planos de execução para consultas SQL sobre tabelas que serão fornecidos. Será bastante utilizado o comando EXPLAIN ANALYZE, que permite visualizar todas as etapas envolvidas no processamento de uma consulta. Usaremos para isso a tabela [“movies”](https://drive.google.com/file/d/1W6wovSsVu4B0OIo_tsSBBHi8WRKQqnat/view?usp=drive_link).

# Parte 4

O objetivo desta parte do trabalho é experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas.